# K-nn algorithm using SVD pre-processing

As we train our k-nn model on vectors of with a really large dimension, we would want to reduce the dimension and will thus use SVD : Singular Values Decomposition in order to project our vectors in a smaller space, and use cosine similarity as a distance between vectors.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

Let's start by testing how our dataset responds to SVD

In [2]:
#Get the data

#read data
data = pd.read_csv('steam-200k.csv')
#clean data
data.columns = ['id','game','state','hours_played','0']
data = data.drop('0',axis=1)
played_games = data.loc[data['state']=='play']
#Get a dict of games and hours played for each id
played_dict = played_games.groupby('id').apply(lambda g : dict(zip(g['game'], g['hours_played'])))

In [3]:
#Standardize
standardization_dict = dict()
for game_name, s in played_games.groupby('game')['hours_played']:
    standardization_dict[game_name]=dict()
    series = s[s>0.0]  #take only games played
    standardization_dict[game_name]['average'] = series.mean()
    if series.std() > 0 : #for some games that are not much played, std is 0 which creates errors
        standardization_dict[game_name]['std'] = series.std()
    else:
        standardization_dict[game_name]['std'] = 1e-8

def standardize(game, hours):
    return (hours - standardization_dict[game]['average'])/standardization_dict[game]['std']

standardized = played_games.copy()

standardized['hours_played'] = standardized.apply(lambda x : standardize(x.game, x.hours_played),axis=1)


To apply SVD, we want to project our players in a smaller space depending on the games they played, so want the players in columns and the games in rows

In [6]:
svd_encoding = standardized.drop_duplicates(subset=['game','id'])
svd_encoding = svd_encoding.pivot(index='game', columns='id', values='hours_played')
svd_encoding = svd_encoding.fillna(0)

svd_encoding.head()

id,5250,76767,86540,144736,181212,229911,298950,381543,547685,554278,...,309228590,309255941,309262440,309265377,309404240,309434439,309554670,309626088,309824202,309903146
game,,,,,,,,,,,,,,,,,,,,,
007 Legends,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Second Ninja,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
u, s, vh = np.linalg.svd(svd_encoding)

In [8]:
k = 5 #number of components kept
uk = u[:,:k]
sk = np.diag(s[:k])
vhk = vh[:k,:]

hours_KD = svd_encoding.T@uk

In [9]:
hours_KD = pd.DataFrame(hours_KD)
hours_KD

,0,1,2,3,4
id,,,,,
5250,-0.338704,-0.241455,0.018510,-0.180361,0.013846
76767,-0.004468,-0.045326,0.625901,0.118420,-0.108568
86540,-0.000450,-0.021970,-0.042716,0.213445,-0.019004
144736,0.000483,-0.000141,0.009455,0.002186,-0.000771
181212,0.000752,0.001373,-0.000305,0.068564,-0.009787
...,...,...,...,...,...
309434439,-0.335904,0.000417,-0.003479,0.002401,0.000676
309554670,-0.000066,0.000018,0.000796,0.000232,0.000218
309626088,0.000569,-0.004437,-0.002730,0.023579,-0.002980


Let's try to find what the components explain !

In [10]:
#Outlier of component 0
played_dict[hours_KD.iloc[:,0].idxmax()]

{'Dota 2': 10442.0,
 'Counter-Strike Global Offensive': 405.0,
 'Rust': 3.4,
 'FreeStyle2 Street Basketball': 2.5,
 'Aura Kingdom': 0.2,
 'Left 4 Dead 2': 0.1}

In [11]:
#Outlier of component 1
played_dict[hours_KD.iloc[:,1].idxmax()]

{'Team Fortress 2': 9640.0}

In [12]:
#Outlier of component 2
played_dict[hours_KD.iloc[:,2].idxmax()]

{'The Elder Scrolls V Skyrim': 1090.0,
 'Mount & Blade Warband': 825.0,
 'Dragon Age Origins': 405.0,
 'Chivalry Medieval Warfare': 340.0,
 'Saints Row The Third': 333.0,
 'Terraria': 327.0,
 'Counter-Strike Source': 316.0,
 'Mass Effect 2': 311.0,
 'Saints Row 2': 308.0,
 'Dota 2': 235.0,
 'Far Cry 3': 234.0,
 'Age of Empires II HD Edition': 227.0,
 'Just Cause 2': 196.0,
 "Assassin's Creed IV Black Flag": 185.0,
 "Assassin's Creed II": 182.0,
 'Portal 2': 178.0,
 'Tropico 4': 170.0,
 'The Witcher 3 Wild Hunt': 154.0,
 'Deus Ex Human Revolution': 147.0,
 'Fallout New Vegas': 139.0,
 'Sleeping Dogs': 139.0,
 'Grand Theft Auto San Andreas': 135.0,
 'BioShock Infinite': 135.0,
 'Max Payne 3': 126.0,
 'Need for Speed Hot Pursuit': 122.0,
 'Wolfenstein The New Order': 122.0,
 'The Elder Scrolls III Morrowind': 116.0,
 'Far Cry 3 Blood Dragon': 107.0,
 'Grand Theft Auto V': 103.0,
 'Worms Reloaded': 91.0,
 'From Dust': 91.0,
 'Middle-earth Shadow of Mordor': 89.0,
 "Assassin's Creed Revelat

In [13]:
#Outlier of component 3
played_dict[hours_KD.iloc[:,3].idxmax()]

{'Counter-Strike Global Offensive': 663.0,
 "Sid Meier's Civilization V": 550.0,
 'Total War SHOGUN 2': 212.0,
 'Total War ROME II - Emperor Edition': 198.0,
 'Dungeon Defenders': 195.0,
 'Age of Empires Online': 168.0,
 'XCOM Enemy Unknown': 126.0,
 'Empire Total War': 125.0,
 'Might & Magic Heroes VI': 118.0,
 "Assassin's Creed IV Black Flag": 94.0,
 'Alien Swarm': 83.0,
 "Assassin's Creed II": 79.0,
 "Assassin's Creed Brotherhood": 73.0,
 'Terra Incognita ~ Chapter One The Descendant': 61.0,
 'Warlock - Master of the Arcane': 55.0,
 'Phantom Breaker Battle Grounds': 53.0,
 'Soul Gambler': 48.0,
 'Supreme Commander': 47.0,
 'The Incredible Adventures of Van Helsing II': 43.0,
 'Titan Quest Immortal Throne': 40.0,
 'Orcs Must Die! 2': 39.0,
 "Assassin's Creed Revelations": 38.0,
 'Relic Hunters Zero': 38.0,
 "Tom Clancy's Ghost Recon Advanced Warfighter": 37.0,
 'Warhammer 40,000 Dawn of War II - Chaos Rising': 35.0,
 'Napoleon Total War': 34.0,
 'Warhammer 40,000 Dawn of War II  Retr

In [14]:
#Outlier of component 4
played_dict[hours_KD.iloc[:,4].idxmax()]

{'The Elder Scrolls V Skyrim': 1090.0,
 'Mount & Blade Warband': 825.0,
 'Dragon Age Origins': 405.0,
 'Chivalry Medieval Warfare': 340.0,
 'Saints Row The Third': 333.0,
 'Terraria': 327.0,
 'Counter-Strike Source': 316.0,
 'Mass Effect 2': 311.0,
 'Saints Row 2': 308.0,
 'Dota 2': 235.0,
 'Far Cry 3': 234.0,
 'Age of Empires II HD Edition': 227.0,
 'Just Cause 2': 196.0,
 "Assassin's Creed IV Black Flag": 185.0,
 "Assassin's Creed II": 182.0,
 'Portal 2': 178.0,
 'Tropico 4': 170.0,
 'The Witcher 3 Wild Hunt': 154.0,
 'Deus Ex Human Revolution': 147.0,
 'Fallout New Vegas': 139.0,
 'Sleeping Dogs': 139.0,
 'Grand Theft Auto San Andreas': 135.0,
 'BioShock Infinite': 135.0,
 'Max Payne 3': 126.0,
 'Need for Speed Hot Pursuit': 122.0,
 'Wolfenstein The New Order': 122.0,
 'The Elder Scrolls III Morrowind': 116.0,
 'Far Cry 3 Blood Dragon': 107.0,
 'Grand Theft Auto V': 103.0,
 'Worms Reloaded': 91.0,
 'From Dust': 91.0,
 'Middle-earth Shadow of Mordor': 89.0,
 "Assassin's Creed Revelat

The different components seem to represent extremely accurately the kind of players in the dataset.  
Indeed, a huge part of the players only plays 3 popular games : Dota 2, Team Fortress 2 and Counter-Strike Global offensive.  
The two first components enables to separate these players, and the next ones represent players that like other types of games.  
4th component seems to represent FPS/strategy games players, and 3rd and 5th one represent RPGs players which are two huge types of players.  
This methods thus seems to work impressively well while necessiting only a few calculations.  

Let's try firstly to do it on players that played at least 3 games who are more interesting for us, and then implement it in our model !

In [6]:
played_dict_3 = played_dict.loc[played_dict.map(len)>=3]

In [16]:
#Create vectors of hours played 
hours_encoded_3 = played_dict_3.apply(pd.Series)
#Replace NaN values by 0 : a game not in the dict has never been played
hours_encoded_3 = hours_encoded_3.fillna(0)
#Sort by name of games
hours_encoded_3 = hours_encoded_3.reindex(sorted(hours_encoded_3.columns),axis=1)

def standardize(c):
            m = c.mean()
            if c.std() > 0:
                std = c.std()
            else:
                std = 1e-8
            return (c-m)/std

hours_encoded_3 = hours_encoded_3.apply(lambda column : standardize(column),axis=0)

svd_encoded_3 = hours_encoded_3.transpose()

u, s, vh = np.linalg.svd(svd_encoded_3)

n_components = 5 #number of components kept
uk = u[:,:n_components]
sk = np.diag(s[:n_components])
vhk = vh[:n_components,:]

hours_KD = svd_encoded_3.T@uk
hours_KD = pd.DataFrame(hours_KD)
hours_KD

,0,1,2,3,4
id,,,,,
5250,0.463470,0.172731,0.562478,-0.102481,0.231477
76767,0.509715,0.156981,0.728053,-0.339869,0.080634
86540,0.275617,-0.245484,-0.894901,-0.711417,-0.583306
229911,0.534997,0.130914,0.805660,-0.429552,0.060478
298950,0.008285,0.084148,-4.983736,2.795756,-0.479561
...,...,...,...,...,...
306547522,0.536958,0.131165,0.963313,-0.507410,0.086452
306971738,0.543779,0.132301,0.984786,-0.535689,0.090316
308695132,0.536834,0.130116,0.962352,-0.504290,0.086035


In [17]:
#Outlier of component 0
played_dict[hours_KD.iloc[:,0].idxmax()]

{'Mystery PI The Vegas Heist': 18.6,
 'Mystery P.I. The Lottery Ticket': 13.2,
 'Amazing Adventures Around the World': 12.0,
 'Mishap An Accidental Haunting': 6.4,
 'Chuzzle Deluxe': 6.3,
 'The Tiny Bang Story': 6.2,
 "The Wizard's Pen": 4.2,
 'Amazing Adventures The Lost Tomb': 3.3,
 'Big Money! Deluxe': 3.0,
 'Escape Rosecliff Island': 2.0,
 'Talismania Deluxe': 1.9,
 'Dynomite! Deluxe': 1.9,
 'Anno 2070': 1.4,
 'Bejeweled Deluxe': 1.0,
 'Feeding Frenzy 2 Shipwreck Showdown Deluxe': 0.4,
 'Bejeweled 2 Deluxe': 0.3}

Adventure/Puzzle games (games liked by 'everyone' as it is the first component)

In [18]:
#Outlier of component 1
played_dict[hours_KD.iloc[:,1].idxmax()]

{'Counter-Strike Global Offensive': 663.0,
 "Sid Meier's Civilization V": 550.0,
 'Total War SHOGUN 2': 212.0,
 'Total War ROME II - Emperor Edition': 198.0,
 'Dungeon Defenders': 195.0,
 'Age of Empires Online': 168.0,
 'XCOM Enemy Unknown': 126.0,
 'Empire Total War': 125.0,
 'Might & Magic Heroes VI': 118.0,
 "Assassin's Creed IV Black Flag": 94.0,
 'Alien Swarm': 83.0,
 "Assassin's Creed II": 79.0,
 "Assassin's Creed Brotherhood": 73.0,
 'Terra Incognita ~ Chapter One The Descendant': 61.0,
 'Warlock - Master of the Arcane': 55.0,
 'Phantom Breaker Battle Grounds': 53.0,
 'Soul Gambler': 48.0,
 'Supreme Commander': 47.0,
 'The Incredible Adventures of Van Helsing II': 43.0,
 'Titan Quest Immortal Throne': 40.0,
 'Orcs Must Die! 2': 39.0,
 "Assassin's Creed Revelations": 38.0,
 'Relic Hunters Zero': 38.0,
 "Tom Clancy's Ghost Recon Advanced Warfighter": 37.0,
 'Warhammer 40,000 Dawn of War II - Chaos Rising': 35.0,
 'Napoleon Total War': 34.0,
 'Warhammer 40,000 Dawn of War II  Retr

CS + strategy

In [19]:
#Outlier of component 2
played_dict[hours_KD.iloc[:,2].idxmax()]

{'Terraria': 392.0,
 'The Elder Scrolls V Skyrim': 344.0,
 'The Binding of Isaac Rebirth': 303.0,
 'The Binding of Isaac': 133.0,
 'Hero Siege': 109.0,
 'AdVenture Capitalist': 68.0,
 'Clicker Heroes': 66.0,
 'Awesomenauts': 60.0,
 'Borderlands 2': 60.0,
 'Audiosurf': 41.0,
 'You Must Build A Boat': 32.0,
 'Deponia': 30.0,
 'Trine': 22.0,
 'Triple Town': 21.0,
 'Tap Heroes': 21.0,
 'Counter-Strike Global Offensive': 20.0,
 'Trine 2': 19.3,
 'One Finger Death Punch': 16.7,
 'The Albino Hunter': 16.7,
 'QuestRun': 16.2,
 'Antisquad': 16.2,
 'Torchlight II': 15.6,
 'Faerie Solitaire': 15.4,
 "Garry's Mod": 15.0,
 'Gravity Badgers': 14.9,
 'Why So Evil 2 Dystopia': 14.3,
 'Point Perfect': 14.0,
 'Battlepaths': 12.9,
 'Bardbarian': 12.7,
 'Beyond Space': 12.6,
 'Scourge Outbreak': 12.5,
 "Sid Meier's Ace Patrol": 11.9,
 'Torchlight': 11.8,
 'E.Y.E Divine Cybermancy': 11.7,
 'Bejeweled 3': 11.1,
 'Cave Story+': 11.0,
 '4 Elements': 10.9,
 'The Lady': 10.8,
 'Evoland': 10.8,
 'Chaos on Deponi

Adventure

In [20]:
#Outlier of component 3
played_dict[hours_KD.iloc[:,3].idxmax()]

{'Team Fortress 2': 994.0,
 'Kerbal Space Program': 480.0,
 'Gnomoria': 246.0,
 'Left 4 Dead 2': 196.0,
 "Sid Meier's Civilization V": 182.0,
 'Saints Row The Third': 110.0,
 'Fallout New Vegas': 107.0,
 'Tropico 4': 93.0,
 'Age of Wonders III': 84.0,
 'The Elder Scrolls V Skyrim': 71.0,
 'Dragon Age Origins': 67.0,
 'Anno 2070': 64.0,
 'Saints Row IV': 63.0,
 'Prison Architect': 55.0,
 'Tropico 5': 54.0,
 'Cities Skylines': 54.0,
 'Borderlands': 51.0,
 'Awesomenauts': 51.0,
 'FTL Faster Than Light': 51.0,
 'Tropico 3 - Steam Special Edition': 47.0,
 'Two Worlds Epic Edition': 45.0,
 "Sid Meier's Civilization IV Beyond the Sword": 43.0,
 'FINAL FANTASY XIII-2': 43.0,
 'Pixel Piracy': 39.0,
 'Supreme Commander 2': 39.0,
 "Sid Meier's Civilization IV": 39.0,
 'Magicka': 38.0,
 'Risen': 36.0,
 'Mass Effect 2': 34.0,
 'Global Agenda': 33.0,
 'Sins of a Solar Empire Trinity': 33.0,
 'Two Worlds II': 33.0,
 'Skyward Collapse': 32.0,
 'Vampire The Masquerade - Bloodlines': 30.0,
 'Star Wolves

Team Fortress + RPG

In [21]:
#Outlier of component 4
played_dict[hours_KD.iloc[:,4].idxmax()]

{'The Elder Scrolls V Skyrim': 104.0,
 'Two Worlds II': 56.0,
 'Tomb Raider': 46.0,
 'Risen 2 - Dark Waters': 40.0,
 'The Witcher 2 Assassins of Kings Enhanced Edition': 38.0,
 "The Musketeers Victoria's Quest": 36.0,
 'Two Worlds Epic Edition': 31.0,
 'Dungeon Siege III': 27.0,
 'LEGO Harry Potter Years 5-7': 26.0,
 'LEGO The Hobbit': 19.7,
 "Assassin's Creed III": 19.5,
 'Age of Empires III Complete Collection': 18.6,
 'SpellForce 2 - Faith in Destiny': 18.3,
 'Batman Arkham City GOTY': 15.9,
 '12 Labours of Hercules II The Cretan Bull': 14.7,
 'Lara Croft and the Guardian of Light': 13.6,
 'Ballad of Solar': 10.1,
 'Heroes of Hellas 3 Athens': 9.9,
 "Broken Sword 5 - the Serpent's Curse": 8.3,
 'Assassins Creed Chronicles China': 6.6,
 'Grim Legends The Forsaken Bride': 6.3,
 'Enigmatis 2 The Mists of Ravenwood': 5.9,
 'Nightmares from the Deep 3 Davy Jones': 5.9,
 "Melissa K. and the Heart of Gold Collector's Edition": 5.3,
 'Kingdom Tales 2': 5.3,
 "Portal of Evil Stolen Runes Col

RPG/Adventure

As a result, it seems that the components are less discriminating than the previous ones, but it is logical because we have kept players that tend not to be super specialized in 1 or 2 games, and they still can enable us to distinct different types of players.  

Now, let's test with the entire model !

To do so, we first need to redefine a distance metrics, and we will choose cosine similarity as it is the most common metrics in this case.  
To be precise, cosine similarity is not a distance in a mathematical sense, but it is not a problem for K-nn.  

Let's implement the svd projection in the fitting part of our model, and using cosine similarity for predicting neighbors.  

To be able to project the vector for prediction in the space, we will store the uk matrix.

In [4]:
class SteamPredictionModel():
    
    def __init__(self, k_neighbors = 20, n_components = 5):
        self.neigh = NearestNeighbors(n_neighbors=k_neighbors, metric='cosine') #cosine metrics
        self.games_list = []
        self.likeness = None
        self.average_played = {}
        self.n_components = n_components
        self.uk_matrix = None

    def dict_to_likeness(self, dicti):
        d = dicti.copy()
        for game in d.keys():
            if d[game] <= self.average_played[game]:
                d[game]=0
        return d
    
    #We fit the model on a dataset containing ids and dictionnaries of games associated with time played
    def fit(self, data):

        #Firstly we encode the hours played
        hours_encoded = data.apply(pd.Series)
        #Replace NaN values by 0 : a game not in the dict has never been played
        hours_encoded = hours_encoded.fillna(0)
        hours_encoded = hours_encoded.reindex(sorted(hours_encoded.columns),axis=1)
        

        #For each player, we compute the list of game he likes with the time he has played aboved average time played
        non_zero_dict = hours_encoded.replace(0, np.NaN)
        self.average_played = non_zero_dict.mean(axis=0)
        likeness_games = data.map(self.dict_to_likeness)
        #And encode them
        likeness_games_encoded = likeness_games.apply(pd.Series)
        #Replace NaN values by 0 : a game not in the dict has never been played
        likeness_games_encoded = likeness_games_encoded.fillna(0)
        likeness_games_encoded = likeness_games_encoded.reindex(sorted(likeness_games_encoded.columns),axis=1)

        #standardization
        #We standardize each column separately

        def standardize(c):
            m = c.mean()
            if c.std() > 0:
                std = c.std()
            else:
                std = 1e-8
            return (c-m)/std

        hours_encoded = hours_encoded.apply(lambda column : standardize(column),axis=0)


        #we also standardize the likeness
        likeness_games_encoded = likeness_games_encoded.apply(lambda column : standardize(column),axis=0)
        self.likeness = likeness_games_encoded

        #SVD
        svd_encoded = hours_encoded.transpose()

        u, s, vh = np.linalg.svd(svd_encoded)

        uk = u[:,:self.n_components]
        self.uk_matrix = uk

        hours_KD = svd_encoded.T@uk
        hours_KD = pd.DataFrame(hours_KD)


        self.neigh.fit(hours_KD.values)
        
        #Get the list of games
        games_list = list(hours_encoded.columns)
        games_list.sort()
        self.games_list = games_list
        
        
    
    #We predict a certain number of games (maximum) using a dictionnary of games associated with time played
    def predict(self, X_init, recommendations_number_max, print_projection = False):
        #One-hot-encode X
        X = pd.Series(X_init,index=self.games_list).fillna(0)
        
        #Create a vector with all games and a null score
        score = pd.Series({self.games_list[0]:0.0},index=self.games_list).fillna(0)

        #Get the list of games played by X
        already_owned = [self.games_list[index] for index in np.asarray(X).nonzero()[0]]

        #Project X on the svd space
        X_proj = X@self.uk_matrix

        if print_projection:
            print(X_proj)

        
        #Get the neighbors of X
        kneighbors = self.neigh.kneighbors([X_proj])
        kneighbors_distances = kneighbors[0][0]
        kneighbors_indices = kneighbors[1][0]

        for i in range(len(kneighbors_indices)):
            neighbor = kneighbors_indices[i]
            #get the list of liked games
            liked = [self.games_list[index] for index in np.asarray(self.likeness.iloc[neighbor]).nonzero()[0]]
            #Add to each game score (1/d)*l with d the distance between X and the neighbor
            #and l the amount of time played above the average
            for liked_game in liked:
                if liked_game not in already_owned:
                    score[liked_game] = score[liked_game] + 1/kneighbors_distances[i]*self.likeness.iloc[neighbor][liked_game]


        score = score.sort_values(ascending=False)
        return score.iloc[:recommendations_number_max]

Let's test this new model

In [26]:
SPM = SteamPredictionModel(k_neighbors=20,n_components=10)
SPM.fit(played_dict_3)

In [27]:
rpg_player = {'The Witcher 3 Wild Hunt':250,'The Elder Scrolls V Skyrim':224, 'Far Cry':90, 'Fallout 3':70}
SPM.predict(rpg_player,20, print_projection=True)

[ -0.83605786  -1.34510491 -11.46905344   7.45974421  -0.85438224
  -0.60605205  -7.48327774  22.13340478   8.27682105  -5.31599383]


Demonicon                        5382.574187
Project Snowblind                5382.574187
Ys The Oath in Felghana          5382.574187
Gothic II Gold Edition           5382.574187
Daikatana                        5382.574187
Deus Ex Revision                 5382.574187
Risen 3 - Titan Lords            4958.716019
Betrayer                         4073.974856
Mount & Blade Warband            4068.799831
Ys Origin                        4039.761703
Narcissu 1st & 2nd               4014.259725
Call of Juarez Bound in Blood    3850.615756
Dishonored                       3848.916903
Deus Ex Human Revolution         3563.041985
From The Depths                  3508.362056
Savage Lands                     3469.897765
War of the Roses Balance Beta    3459.380583
Another World                    3443.364368
Cryostasis                       3422.368771
Divinity II Developer's Cut      3165.952468
dtype: float64

The results are different from the previous algorithm but still interesting and rather in the theme !

In [28]:
fps_player = {'Counter-Strike Global Offensive':500,'Call of Duty Modern Warfare 3':70,'Call of Duty World at War':120}
SPM.predict(fps_player,20, print_projection=True)


[-1.42778365  0.89995742 -0.65809293  1.73439864 -1.47011286  0.72822026
 -4.84661419  4.98072946 -5.46006107  1.74444726]


The Cat Lady                                1604.230660
DiRT 3 Complete Edition                     1484.860355
WRC 4 FIA WORLD RALLY CHAMPIONSHIP          1484.778880
Dead Effect                                 1481.958635
The Novelist                                1481.958635
BlazBlue Calamity Trigger                   1481.958635
Serious Sam Classic The Second Encounter    1481.958635
FOTONICA                                    1481.958635
Dementium II HD                             1481.958635
Neverending Nightmares                      1481.958635
Arma Tactics                                1411.511965
Killer is Dead                              1298.967807
Nihilumbra                                  1152.119486
Serious Sam Classic The First Encounter      940.242500
Deadly Premonition The Director's Cut        896.806842
Painkiller Hell & Damnation                  831.870374
TypeRider                                    824.934858
Wickland                                     808

# Evaluation

We will use the last method used for the previous k-nn algorithm for evaluation.

In [17]:
def train_validation_test_split(data, validation_size = 0.1, test_size=0.1):
    
    #shuffle
    shuffled = data.sample(frac=1, random_state=42) #set seed for reproducability
    
    #split train and test
    separator1 = len(shuffled) - int(len(shuffled)*(test_size+validation_size))
    separator2 = len(shuffled) - int(len(shuffled)*(test_size))
    X_train = shuffled.iloc[:separator1]
    X_validation = shuffled.iloc[separator1:separator2]
    X_test = shuffled.iloc[separator2:]
    
    return (X_train,X_validation,X_test)

X_train, X_validation, X_test = train_validation_test_split(played_dict)

In [23]:
#Create and fit the model
SPM = SteamPredictionModel(k_neighbors=20,n_components=5)
SPM.fit(X_train)

In [24]:
from tqdm import tqdm

#Create vectors of hours played 
hours_encoded = played_dict.apply(pd.Series)
#Replace NaN values by 0 : a game not in the dict has never been played
hours_encoded = hours_encoded.fillna(0)
#We drop the ids because they are not useful anymore
hours_encoded = hours_encoded.reset_index(drop=True)
#Sort by name of games
hours_encoded = hours_encoded.reindex(sorted(hours_encoded.columns),axis=1)
hours_encoded.head()

non_zero_dict = hours_encoded.replace(0, np.NaN)
average_played = non_zero_dict.mean(axis=0)

#Evaluate the model
true_positives = 0
false_positives = 0

for player in tqdm(X_test.loc[X_test.map(len)>=5]):
    
    player_games = len(player)
    
    games_to_predict = 1 # we search only 1 game
    
    games_for_prediction = {}
    games_removed = []
    n = 0
    for (name,time) in list(player.items()):
        if n < games_to_predict and time >average_played[name]: #the game is liked
            games_removed.append(name)
            n += 1
        else:
            games_for_prediction[name]=time
    
    prediction = list(SPM.predict(games_for_prediction,10).index) #we predict 10 games
    
    if len(games_removed) > 0:
        if games_removed[0] in prediction:
            true_positives += 1
        else:
            false_positives += 1

precision = true_positives/(true_positives+false_positives)

100%|██████████| 248/248 [07:54<00:00,  1.91s/it]


In [25]:
print(true_positives,false_positives,precision)

0 220 0.0


In [26]:
print(precision/len(list(hours_encoded.columns)))

0.0


It seems that this model performs very poorly, even thought the components really well described the types of players.  
I would say that it means that this method has a real potential, but needs more theoretical and precise approach to be used.  

We could try to improve the hyperparameters k_neighbors and n_components with a grid search method for instance, but it would firstly require a model improvement.
